1) El servicio meteorológico registra datos del
tiempo para todas las ciudades donde posee
una base de medición.
Esta información se encuentra en dos RDD.
En el primero se tiene información de las
bases de medición: (ID_BASE, NOMBRE,
PCIA, CIUDAD, LAT, LON).
El segundo posee información sobre las
mediciones en sí: (TIMESTAMP, ID_BASE,
TEMPERATURA, HUMEDAD, PRESIÓN,
DIRECCIÓN VIENTO, VELOCIDAD VIENTO).

Se desea obtener un reporte para las bases de
la Provincia de Buenos Aires. El mismo debe
informar los ID y nombre de bases de
medición que hayan registrado una variación
de temperatura promedio mensual mayor al
30% en el año 2018 (dada la temperatura
promedio de un mes, esta se debe comparar
con el promedio del mes anterior, para
determinar la variación porcentual).
Resolver utilizando el API de RDD de
PySpark, dando el reporte en un RDD. (***) (
15pts)

In [7]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

aux = sc.textFile('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2019-1C-1Op.csv')
basesRdd = aux.map(lambda x: x.split(","))
basesRdd = basesRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1], x[2], x[3],
                                 pd.to_numeric(x[4], 'coerce'), pd.to_numeric(x[5], 'coerce')))

aux = sc.textFile('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2019-1C-1OpMediciones.csv')
meditionsRdd = aux.map(lambda x: x.split(","))
meditionsRdd = meditionsRdd.map(lambda x: (pd.to_datetime(x[0], 'coerce', format='%Y-%m-%d'),
                                           pd.to_numeric(x[1], 'coerce'), pd.to_numeric(x[2], 'coerce'),
                                           pd.to_numeric(x[3], 'coerce'), pd.to_numeric(x[4], 'coerce'),
                                            x[5], pd.to_numeric(x[6], 'coerce')))

buenosAiresBases = basesRdd.filter(lambda x: x[2] == 'Buenos Aires').map(lambda x: (x[0], x[1]))

monthlyAveragePerBase = meditionsRdd.map(lambda x: (x[1], (x[0], x[2]))).join(buenosAiresBases).\
            filter(lambda x: x[1][0][0].year == 2018).map(lambda x: ((x[0], x[1][0][0].month, x[1][1]), (x[1][0][1], 1))) \
            .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[0]/x[1])

#groupedData = monthlyAveragePerBase.map(lambda x: ((x[0][0], x[0][2]), (x[0][1], x[1]))).groupByKey()

'''
def f(x):
    monthly_temp = list(x)
    monthly_temp.sort()
    for i in range(0, len(monthly_temp) - 1):
        if (monthly_temp[i+1][0] - monthly_temp[i][0]) == 1 :
            if (abs(monthly_temp[i][1] - monthly_temp[i+1][1]) / monthly_temp[i][1]) > 0.3:
                return True
    return False


groupedData.mapValues(f).filter(lambda x: x[1] == True).map(lambda x: (x[0][0], x[0][1])).collect()
'''

def f(x,y):
    if x[0] is not None:
        return x[0], y[1]
    return y[0], x[1]

monthlyAveragePerBase.flatMap(lambda x: ( (x[0], (x[1], None)), ((x[0][0], x[0][1] - 1, x[0][2]), (None, x[1])) )) \
                    .reduceByKey(f).filter(lambda x: (x[1][0] is not None) & (x[1][1] is not None))\
                    .filter(lambda x: abs((x[1][0] - x[1][1]) / x[1][0]) > 0.3).map(lambda x: (x[0][0], x[0][2])) \
                    .distinct().collect()
#Opinion: la solucion sin el groupBy me parece una verga y hasta mas ineficiente, pero parece que ellos prefieren eso

[((5, 4, 'oucmkxnka'), (23.77777777777778, 26.0)),
 ((5, 8, 'oucmkxnka'), (25.46153846153846, 29.4)),
 ((3, 7, 'dcqyupw'), (20.6, 12.375)),
 ((3, 11, 'dcqyupw'), (23.571428571428573, 18.142857142857142)),
 ((3, 3, 'dcqyupw'), (14.75, 22.571428571428573)),
 ((3, 6, 'dcqyupw'), (22.0, 20.6)),
 ((3, 10, 'dcqyupw'), (23.857142857142858, 23.571428571428573)),
 ((3, 2, 'dcqyupw'), (28.0, 14.75)),
 ((5, 9, 'oucmkxnka'), (29.4, 14.454545454545455)),
 ((5, 1, 'oucmkxnka'), (19.642857142857142, 21.733333333333334)),
 ((5, 5, 'oucmkxnka'), (26.0, 27.5)),
 ((3, 1, 'dcqyupw'), (24.0, 28.0)),
 ((3, 5, 'dcqyupw'), (17.1, 22.0)),
 ((3, 9, 'dcqyupw'), (15.9, 23.857142857142858)),
 ((5, 6, 'oucmkxnka'), (27.5, 16.375)),
 ((5, 2, 'oucmkxnka'), (21.733333333333334, 13.166666666666666)),
 ((5, 10, 'oucmkxnka'), (14.454545454545455, 14.375)),
 ((5, 3, 'oucmkxnka'), (13.166666666666666, 23.77777777777778)),
 ((5, 7, 'oucmkxnka'), (16.375, 25.46153846153846)),
 ((5, 11, 'oucmkxnka'), (14.375, 15.6666666666666